In [1]:
import numpy as np
import pandas as pd
import datasetBuilder
import tests
import tools
import spectral_similarity

Include Basic Tests from all steps of process

In [2]:
#ppm

#print ppm for a given precursor and ppm discrepancy
#all precursors within the output below of the precursor are within X ppm 
tools.ppm(400,4)

0.0015999999999999999

Test to see that our similarity methods match the results from original paper with test specs:

Unweighted should be 0.9826
Weighted should be 0.8984

In [3]:
#verify all similarity score results are the same under default settings as orig paper
#the result should read identical to below markdown cell
#show results for 'unweighted entropy' and 'entropy' here as defined in orig paper

spec_query = np.array([[69.071, 7.917962], [86.066, 1.021589], [86.0969, 100.0]], dtype=np.float32)
spec_reference = np.array([[41.04, 37.16], [69.07, 66.83], [86.1, 999.0]], dtype=np.float32)

#no weighting but normal cleaning scheme first
method='entropy'
spec_query = tools.clean_spectrum(spec_query)
spec_reference = tools.clean_spectrum(spec_reference)
merged = tools.match_peaks_in_spectra(spec_query, spec_reference, ms2_da=0.05)
similarity = 1- spectral_similarity.distance(merged,method)
print(f"unweighted_{method} similarity: {similarity}")

#now do weighting scheme from original 
spec_query = tools.weight_intensity(spec_query, power='orig')
spec_reference = tools.weight_intensity(spec_reference, power='orig')

merged = tools.match_peaks_in_spectra(spec_query, spec_reference, ms2_da=0.05)
similarity = 1- spectral_similarity.distance(merged,method)
print(f"{method} similarity: {similarity}")


unweighted_entropy similarity: 0.9826668214969606
entropy similarity: 0.8984397739652141


Now we will go through the dataset creation proces to make sure all steps look sensible. This is how we gather data for testing individual metrics and training/evaluating models

In [4]:
query = pd.read_pickle('/Users/jonahpoczobutt/projects/raw_data/db_csvs/metlin_highres.pkl')
target = pd.read_pickle('/Users/jonahpoczobutt/projects/raw_data/db_csvs/nist14_highres.pkl')
samp=query.iloc[0:1]

User can toggle the parameters of create_matches_df in order to see how the potential matches change

Compare this result to doing filtering manually

In [20]:
ppm_window = 40
max_rows_per_query=None
max_len=1e4
Adduct_match=True

matches = datasetBuilder.create_matches_df_new(samp, target, ppm_window, max_rows_per_query,max_len, Adduct_match)

#first do manual ppm filter
matches_manual = target[(abs(samp.iloc[0]["precursor"] - target["precursor"])< tools.ppm(samp.iloc[0]["precursor"], ppm_window))]

#then do adduct match
matches_manual=matches_manual[matches_manual['precursor_type']==samp.iloc[0]['precursor_type']]

print(f"length matches: {len(matches_manual)==len(matches)}")

total number of query spectra considered: 33
total inchicores seen: 1
length matches: True


Once matches df is made, we can grab similarity metrics

In [6]:
sim_methods = ['entropy']
samp_dataset = datasetBuilder.create_model_dataset(matches, sim_methods)

And Verify that Entropy is the same as What appears in dataset when we calc separately...same protocol as before


In [12]:
spec_query  = matches.iloc[0]['query']
spec_reference  = matches.iloc[0]['target']

precursor_query=matches.iloc[0]['precquery']
prectarget = matches.iloc[0]['prectarget']

#remove precursors
below_prec_indices = np.where(
        spec_query[:, 0] < (precursor_query - tools.ppm(precursor_query, 3))
    )

spec_query = spec_query[below_prec_indices]


method='entropy'
spec_query = tools.clean_spectrum(spec_query)
spec_reference = tools.clean_spectrum(spec_reference) 
spec_query = tools.weight_intensity(spec_query, power='orig')
spec_reference = tools.weight_intensity(spec_reference, power='orig')
merged = tools.match_peaks_in_spectra(spec_query, spec_reference, ms2_da=0.05)
similarity = 1- spectral_similarity.distance(merged,method)
print(f"{method} similarity: {similarity}")
print(f"samp dataset value: {samp_dataset.iloc[0,-2]}")



entropy similarity: 0.18094230249677656
samp dataset value: 0.18094230249677656
